In [1]:
import numpy as np
import itertools
from timeit import default_timer as timer
from IPython.core.debugger import set_trace

In [2]:
def get_input(name):
    with open(f'{name}.txt') as f:
        return f.read()

# Part 1

In [3]:
class Rock:
    rock_shapes = [ 
        [[3, 3, 3, 3], [0, 1, 2, 3]], # I, lying down
        [[1, 2, 2, 2, 3], [1, 0, 1, 2, 1]], # plus
        [[1, 2, 3, 3, 3], [2, 2, 2, 1, 0]], # backwards L
        [[0, 1, 2, 3], [0, 0, 0, 0]] , # I, standing up
        [[2, 2, 3, 3], [0, 1, 0, 1]]  , # box
          ]
    rock_heights = [1, 3, 3, 4, 2]
    
    def __init__(self, y, rock_shape):
        self.x = 2
        self.y = y
        self.shape = rock_shape
    
    def max_y(self):
        return self.y + HITBOX_SIZE - self.rock_heights[self.shape]
         
    def image(self):
        return zip(*self.rock_shapes[self.shape])

In [4]:
class Cave:
    n_rocks = 0
    state = 'active'
    height_outside_of_view = 0
    rolled_field = None
    n_rocks_at_last_update = 0
    current_rock_at_last_update = None
    
    def __init__(self, height, width, rock_limit, jet_streams):
        self.height = height
        self.current_height = height
        self.width = width
        self.field = np.zeros([height+1, width+2])
        self.rock_limit = rock_limit
        self.rock_cycle = itertools.cycle([0, 1, 2, 3, 4])
          
    def intersects(self):
        intersection = False
        for i,j in self.rock.image():
            if i + self.rock.y > self.height - 1 or \
                    j + self.rock.x > self.width - 1 or \
                    j + self.rock.x < 0 or \
                    self.field[i + self.rock.y][j + self.rock.x] > 0:
                intersection = True
        return intersection
    
    def new_rock(self):
        self.rock = Rock(self.current_height - HITBOX_SIZE - 3, next(self.rock_cycle))
        
    def freeze(self):
        for i,j in self.rock.image():
            self.field[i + self.rock.y][j + self.rock.x] = 1
        
        self.n_rocks += 1
        self.current_height = min(self.current_height, self.rock.max_y())
        
        if self.current_height < 2*HITBOX_SIZE:
            self.update_field()
            
        if self.n_rocks == self.rock_limit:
            self.state = 'finished'
        else:
            self.new_rock()
            
    def fall_down(self):
        self.rock.y += 1
        if self.intersects():
            self.rock.y -= 1
            self.freeze()
            return True
        return False

    def go_side(self, jet_direction):
        dx = -1 if jet_direction == '<' else 1
        old_x = self.rock.x
        self.rock.x += dx
        if self.intersects():
            self.rock.x = old_x
    
    
    def get_col_heights(self):
        nz = np.argwhere(self.field)
        lowest_y_for_x = np.zeros(CAVE_WIDTH)
        for x in range(CAVE_WIDTH):
            if x in (cols_w_x := nz[:, 1]):
                lowest_y_for_x[x] = nz[cols_w_x == x][:,0].min()
            else: 
                lowest_y_for_x[x] = self.height
        return lowest_y_for_x
    
    def update_field(self):
        lowest_y_for_x = self.get_col_heights()
        maxy = lowest_y_for_x.max()

        to_roll = int(self.height - maxy)
        self.field = np.roll(self.field, to_roll, axis=0)
        self.field[:to_roll, :] = 0
        
        self.height_outside_of_view += to_roll
        self.current_height += to_roll
        
        
        
        if (self.current_rock_at_last_update == self.rock.shape) & (self.field == self.rolled_field).all():
            # cycle detected
            print('cycle found in field update')
            n_rocks_in_cycle = self.n_rocks - self.n_rocks_at_last_update
            n_rocks_to_go = self.rock_limit - self.n_rocks
            n_whole_cycles = n_rocks_to_go//n_rocks_in_cycle
            self.height_outside_of_view += to_roll*n_whole_cycles
            self.n_rocks += n_rocks_in_cycle*n_whole_cycles
        else:
            self.rolled_field = self.field.copy()
            self.n_rocks_at_last_update = self.n_rocks
            self.current_rock_at_last_update = self.rock.shape
        
        
    def get_total_height(self):
        return self.height_outside_of_view + self.height - self.current_height

In [62]:
def run_cave_sim(max_height, CAVE_WIDTH, max_rocks, jet_streams):
    c = Cave(max_height, CAVE_WIDTH, max_rocks, jet_streams)
    c.new_rock()
    
    height_at_last_check = 0
    
    jet_cycle = zip( itertools.cycle( range(len(jet_streams)) ), itertools.cycle( jet_streams )) 
    while not c.state == 'finished':
        i_jet, jet_direction = next(jet_cycle)
        
        c.go_side(jet_direction)
        did_freeze = c.fall_down()
        
        # THE c.rock.shape < 2 I do not understand fully, but it works. Origininally I thought it needed to be 0, so that we check
        # for cycles when we are at the start of both the jet cycle and the rock cycle. 0 works for my input but not for the test
        # For the test, it needs to equal 1. However, weirdly, if I catch both 1 and 0, both input and test work.....
        if (i_jet == 0) and (c.rock.shape < 2):
            delta_h = c.get_total_height() - height_at_last_check
            
            if delta_h > 0:
                if ('saved_field' in locals()) and \
                   (c.field[c.current_height:c.current_height + delta_h, :] == saved_field[saved_current_height:saved_current_height + delta_h, :]).all():
                    
                    print('cycle found in jet cycle')
                    n_rocks_in_cycle = c.n_rocks - n_rocks_at_last_check
                    n_rocks_to_go = c.rock_limit - c.n_rocks
                    n_whole_cycles = n_rocks_to_go//n_rocks_in_cycle
                    
                    c.height_outside_of_view += delta_h*n_whole_cycles
                    c.n_rocks += n_rocks_in_cycle*n_whole_cycles
                else:               
                    height_at_last_check = c.get_total_height() 
                    rock_shape_at_last_check = c.rock.shape
                    n_rocks_at_last_check = c.n_rocks
                    saved_current_height = c.current_height
                    saved_field = c.field.copy()
    return c

In [ ]:
CAVE_WIDTH = 7
HITBOX_SIZE = 4
rock_limit = 2022
max_height = 1500*HITBOX_SIZE

jet_streams = get_input('test')

c = run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)

In [ ]:
c.get_total_height()

3068

In [ ]:
CRT = ''''''
for row in c.field:
    if any(row):
        CRT += '|'
        for pt in row:        
            CRT += '#' if pt else '.'
        CRT += '|\n'
print(CRT)

## Part 2

In [64]:
CAVE_WIDTH = 7
HITBOX_SIZE = 4
rock_limit = 1000000000000
max_height = 2000*HITBOX_SIZE

jet_streams = get_input('input')

c = run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)

cycle found in jet cycle


In [58]:
c.get_total_height()

1542941176480

## Profiling

In [ ]:
n_rocks_in_cycle

In [ ]:
%load_ext line_profiler

In [ ]:
CAVE_WIDTH = 7
HITBOX_SIZE = 4
rock_limit = 100000
max_height = 1500*HITBOX_SIZE

jet_streams = get_input('test')

%lprun -f Cave.intersects run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)

In [ ]:
CAVE_WIDTH = 7
HITBOX_SIZE = 4
rock_limit = 100000
max_height = 1500*HITBOX_SIZE

jet_streams = get_input('test')

%lprun -f Cave.intersects run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)

In [ ]:
times = []
for rock_limit in [1, 10, 100, 1000, 10000, 100000]:
    if c in locals():
        del c
    t0 = timer()
    CAVE_WIDTH = 7
    HITBOX_SIZE = 4
    # rock_limit = 1000000000000
    max_height = 2500*HITBOX_SIZE

    jet_streams = get_input('test')

    c = run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)
    times.append(timer()-t0)

In [ ]:
# 13 years runtime. Probably need to find another way